In [ ]:
import evo
import numpy as np
import pandas as pd
import csv

from matplotlib.pyplot import figure, show, subplots
from matplotlib import pyplot as plt
from labellines import *
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from matplotlib.lines import Line2D

#import yaml
from ruamel.yaml import YAML
yaml = YAML()
yaml.preserve_quotes = True  # Keep quotes if they exist
yaml.indent(mapping=2, sequence=4, offset=2)  # Preserve indentation
yaml.representer.add_representer(
    float, lambda dumper, data: dumper.represent_scalar("tag:yaml.org,2002:float", str(data))
)

# Only fo2 dependent

In [ ]:
data = []
fo2_range = np.linspace(-4,4,20)
ATOMIC_H_value = 100

for i in fo2_range:
    # Load YAML file
    with open("../config/env.yaml", "r") as file:
        config = yaml.load(file) or {}

    # Modify values
    config["FO2_buffer_START"] = float(i)
    config["ATOMIC_H"] = ATOMIC_H_value

    # Write back to YAML file
    with open("../config/env.yaml", "w") as file:
        yaml.dump(config, file)

    # run evo
    evo.run_evo("../config/chem.yaml", "../config/env.yaml", "../config/output.yaml", folder="../output")
    data.append(np.genfromtxt('../output/dgs_output_basalt_COHSN_closed_1473K.csv',skip_header=1,delimiter=',')[-1][9:19])
data = np.array(data)

In [ ]:
volatile_labels = ['H2O', 'H2', 'O2', 'CO2', 'CO', 'CH4', 'SO2', 'H2S', 'S2', 'N2']

fig = figure(figsize=(8,6))
frame1 = fig.add_subplot(1,1,1)
for i in range(len(volatile_labels)):
    frame1.plot(fo2_range,data[:,i],label=volatile_labels[i])
frame1.set_xlabel("Mantle $f$O$_2$, $\Delta$ IW")
frame1.set_ylabel('Mole fraction')
frame1.set_ylim(1E-3)
frame1.set_yscale('log')
frame1.set_title("Volcanic gas output for varying redox states (H/C mantle mass ratio = 0.5)")
labelLines(plt.gca().get_lines())
filepath = "../output/figures/EVo_1dplot_H" + str(ATOMIC_H_value) + ".png"
fig.savefig(filepath)
show()

# fo2 and C/H dependent

In [ ]:
data_mole = []
data_weight = []

# load fo2 and H inputs
with open("../config/masterconfig.yaml", "r") as file:
    EVo_config = yaml.load(file) or {}
fo2_range = np.linspace(EVo_config['FO2_start'],EVo_config['FO2_stop'],EVo_config['FO2_steps'])
H_range = np.array([2000,200,40])
CH_ratios = 200/H_range

for i in fo2_range:
    for j in H_range:
        # Load YAML file
        with open("../config/env.yaml", "r") as file:
            config = yaml.load(file) or {}

        # Modify fo2 and H values
        config["FO2_buffer_START"] = float(i)
        config["ATOMIC_H"] = float(j)

        # Write back to YAML file
        with open("../config/env.yaml", "w") as file:
            yaml.dump(config, file)

        # run evo
        evo.run_evo("../config/chem.yaml", "../config/env.yaml", "../config/output.yaml", folder="../output")
        
        # write data
        data_mole.append(np.genfromtxt('../output/dgs_output_basalt_COHSN_closed_1473K.csv',skip_header=1,delimiter=',')[-1][9:19])
        data_weight.append(np.genfromtxt('../output/dgs_output_basalt_COHSN_closed_1473K.csv',skip_header=1,delimiter=',')[-1][19:29])

data_mole = np.array(data_mole)
data_weight = np.array(data_weight)

In [ ]:
# get fo2, C/H coordinates:
coords = []
for i in fo2_range:
    for j in CH_ratios:
        coords.append([np.round(i,2),np.round(j,2)])

def write_data(filepath,data):
    with open(filepath, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['fo2','C/H','H2O', 'H2', 'O2', 'CO2', 'CO', 'CH4', 'SO2', 'H2S', 'S2', 'N2'])
        for i in range(len(data)):
            string = [str(coords[i][0]),str(coords[i][1])]
            for j in data[i]:
                string.append(str(j))
            writer.writerow(string)

write_data('../output/atmospheres_mole.csv', data_mole)
write_data('../output/atmospheres_weight.csv', data_weight)

In [ ]:
data_mole = np.genfromtxt("../output/atmospheres_mole.csv",skip_header=1,delimiter=",")[:,2:]
data_weight = np.genfromtxt("../output/atmospheres_weight.csv",skip_header=1,delimiter=",")[:,2:]

# order of volatiles is same as in EVo. Colours are assigned by me. 
volatile_labels = ['H2O', 'H2', 'O2', 'CO2', 'CO', 'CH4', 'SO2', 'H2S', 'S2', 'N2']
volatile_colours = ["blue","red","gray","cyan","green","gray","orange","pink","purple","gray"]

def savefigures(data,filename,plot_title):
   mpl.rcParams['figure.dpi'] = 800
   rasterplot = figure(figsize=(8,6))

   # setup custom order for rasterplot, since the subplots go left->right, up->down
   # but we need to go down->up, left->right
   custom_order = [9,5,1,10,6,2,11,7,3,12,8,4]

   # do plots
   for i in range(len(data)):
      frame = rasterplot.add_subplot(3,4,custom_order[i])
      frame.pie(data[i], labels=None,
         colors=volatile_colours)

   # add axes to the plot (I feel like I over-engineered this)
   master_ax = rasterplot.add_axes([0.1, 0.1, 0.8, 0.8])
   master_ax.patch.set_alpha(0)
   master_ax.set_zorder(-10)

   master_ax.spines['top'].set_visible(False)
   master_ax.spines['right'].set_visible(False)
   master_ax.spines['left'].set_linewidth(1.5)
   master_ax.spines['bottom'].set_linewidth(1.5)

   y_centers = [0.83, 0.50, 0.17] 
   master_ax.set_ylim(0, 1)
   master_ax.set_yticks(y_centers)
   master_ax.set_yticklabels(["5", "1", "0.1"])

   x_centers = [0.135, 0.365, 0.635, 0.865]
   master_ax.set_xlim(0, 1)
   master_ax.set_xticks(x_centers)
   master_ax.set_xticklabels(["-4", "-1.3", "1.3", "4"])

   master_ax.tick_params(labelsize=12)

   # add legend
   custom_lines = []
   for i in volatile_colours:
      custom_lines.append(Line2D([0], [0], color=i, lw=4))

   rasterplot.legend(custom_lines,volatile_labels,bbox_to_anchor=(1.05, 0.9))

   # axes labels
   rasterplot.supxlabel('Mantle $f$O$_2$, $\Delta$ IW')
   rasterplot.supylabel('Mantle C/H mass ratio')
   rasterplot.suptitle(plot_title)

   filepath = "../output/figures/" + filename
   plt.savefig(filepath)

savefigures(data_weight,'EVo_3dplot_weightfraction.png',"Molecular contents of volcanic gas parcel (weight fractions)")
savefigures(data_mole,'EVo_3dplot_molefraction.png',"Molecular contents of volcanic gas parcel (mole fractions)")